Imports

In [13]:
from sklearn.model_selection import train_test_split
from keras.models import Sequential
from os import listdir
from os.path import isfile, join
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report
import string
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow import keras
from keras.models import Sequential
from keras.layers import Dense, Activation, Dropout
from keras.preprocessing import text
from keras import utils
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import confusion_matrix

Setup data for model

In [14]:
# Function that decides what symbols to keep
symb_keep = [" ", "!", "?", ".", ","] + list(string.ascii_letters)
str_fix = lambda str: "".join([char.lower() if char in symb_keep else "" for char in str]).replace("‚Äô", "'")

# Load training data
with open('PA3_train.tsv', encoding="utf-8") as f:
    Y_train, X_train = [list(a) for a in list(zip(*[(int(line[0]), str_fix(line[4:])) for line in [line for line in f.readlines() if len(line) > 4] if (line.strip() and line[0] == line[2])]))]
    
with open('PA3_train_final.tsv', encoding="utf-8") as f:
    Y_train_extra, X_train_extra = zip(*[(int(line[0]), str_fix(line[6:])) for line in [line for line in f.readlines() if len(line) > 6] if (line.strip and line[0] == line[2] and line[2] == line[4])])

Y_train += Y_train_extra
X_train += X_train_extra

# Load test data
with open('PA3_test.tsv', encoding="utf-8") as f:
    Y_test, X_test = zip(*[(int(line[0]), str_fix(line[2:])) for line in f.readlines()])

Run and save deep learning model

In [15]:
# Prepare model

# Set input size
max_words = 5000

# Tokenize layers
tokenize = text.Tokenizer(num_words=max_words, char_level=False)
tokenize.fit_on_texts(X_train) # only fit on train

# Convert text to matrix
X_train = tokenize.texts_to_matrix(X_train)
X_test = tokenize.texts_to_matrix(X_test)

encoder = LabelEncoder()
encoder.fit(Y_train)
Y_train = encoder.transform(Y_train)
Y_test = encoder.transform(Y_test)

num_classes = np.max(Y_train) + 1
Y_train = utils.to_categorical(Y_train, num_classes)
Y_test = utils.to_categorical(Y_test, num_classes)

# Set batch size and amount of epochs
batch_size = 16
epochs = 5

# Build the model
model = Sequential()
model.add(Dense(512, input_shape=(max_words,)))
model.add(Activation('relu'))
model.add(Dropout(0.5))
model.add(Dense(num_classes))
model.add(Activation('softmax'))


# Compire and save model
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

model.save('resturant_classifier.h5')
              
history = model.fit(X_train, Y_train, batch_size=batch_size, epochs=epochs,verbose=1, validation_split=0.1)

Epoch 1/5
777/777 [==============================] - 53s 66ms/step - loss: 0.1267 - accuracy: 0.9536 - val_loss: 0.0455 - val_accuracy: 0.9891
Epoch 2/5
 61/777 [=>............................] - ETA: 45s - loss: 0.0323 - accuracy: 0.9887